# Producer

In [7]:
import kafka
from kafka       import KafkaProducer
from kafka.admin import KafkaAdminClient
from kafka.admin import NewTopic
from tqdm        import tqdm

import pandas as pd
import os
import boto3
import json
import time

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [8]:
KAFKA_HOME = '/usr/local/kafka'
KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092']

In [9]:
# producer definition from IP address given before
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

# connect to the cluster to run admin functions
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        api_version=(0,10,2))

In [10]:
kafka_admin.list_topics()

[]

In [11]:
s3_client = boto3.client('s3',
                         endpoint_url='https://cloud-areapd.pd.infn.it:5210',
                         aws_access_key_id='1a4543841b844a88bb3f2eba45764d61',
                         aws_secret_access_key='42e2f16592f54668b8421ecf5ca7ba51',
                         verify=False)

In [12]:
list_obj_contents = s3_client.list_objects(Bucket= 'mapd-minidt-stream')['Contents']
list_obj_contents[0:5]

[{'Key': 'data_000000.txt',
  'LastModified': datetime.datetime(2022, 5, 30, 8, 56, 44, 485000, tzinfo=tzlocal()),
  'ETag': '"0a7dccb174b6732af3c9e040109df403-2"',
  'Size': 29604978,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'PhysicsOfData-students',
   'ID': 'd2e941ce4b324467b6b3d467a923a9bc$d2e941ce4b324467b6b3d467a923a9bc'}},
 {'Key': 'data_000001.txt',
  'LastModified': datetime.datetime(2022, 5, 30, 8, 56, 44, 580000, tzinfo=tzlocal()),
  'ETag': '"777544f2f3b53cc8043791bde33c375a-2"',
  'Size': 29757766,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'PhysicsOfData-students',
   'ID': 'd2e941ce4b324467b6b3d467a923a9bc$d2e941ce4b324467b6b3d467a923a9bc'}},
 {'Key': 'data_000002.txt',
  'LastModified': datetime.datetime(2022, 5, 30, 8, 56, 44, 644000, tzinfo=tzlocal()),
  'ETag': '"d7f465cb61aafab424073b58f4cc1aa1-2"',
  'Size': 30921744,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'PhysicsOfData-students',
   'ID': 'd2e941ce4b324467b6b3d467a9

In [13]:
n = len(list_obj_contents)
for i in range(0,n):
    file_name = list_obj_contents[i]['Key']
    row_data = pd.read_csv(s3_client.get_object(Bucket='mapd-minidt-stream', 
                                                Key=file_name).get('Body'), dtype='str')
    jj = row_data.to_dict('records')
    
    # For loop over file size
    for msg in tqdm(jj):
        # dictionaries creation from dataframe's rows
        # unnecessary floats are cast into ints
        # json row is sent to the Kafka topic 'topic_stream'   
        producer.send('stream', json.dumps(msg).encode('utf-8'))
        time.sleep(0.00001)
    producer.flush()


 81%|████████████████████████████████████▌        | 1064727/1310720 [13:11<03:02, 1345.80it/s]


KeyboardInterrupt: 